In [1]:
##Load metadata from file

import ast

d = {}
nary = {}
with open("test_case-8classes_2properties.txt", 'r') as f:
    for line in f:
        d = (ast.literal_eval(line))
        nary.update({d['id'] : d})

In [2]:
#Filter terms that have children and aspect

filnary = {}
kilnary = {}
nochild = {}

for key, value in nary.items():
    if value.get('children') != []:
        filnary = ({'id' : value['id'], 'children' : value['children']})
        kilnary.update({key : filnary})
    else:
        filnary = ({'id' : value['id'],})
        nochild.update({key : filnary})

In [3]:
#get parents

for key, value in kilnary.items():
    for item in value['children']:
        if nary[item['id']].get('parent') == None:
            nary[item['id']]['parent'] = []
            nary[item['id']]['parent'].append({'id' : key, 'relation' : item['relation']})
        else:
            nary[item['id']]['parent'].append({'id' : key, 'relation' : item['relation']})
            
for key, value in nary.items():
    if value.get('parent') == None:
        value['parent'] = []

In [4]:
#import owlready2

import types
from owlready2 import *
onto = get_ontology("http://test.org/onto.owl")

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [5]:
#define core classes

with onto:
    class therapeutic_agent(Thing):
        namespace = onto
    class biological_process(Thing):
        namespace = onto
    class molecular_function(Thing):
        namespace = onto
    class drug(Thing):
        namespace = onto

In [6]:
def recursive_classmaker_down(term):
    for item in nary[term]['children']:
        with onto:
            nc = types.new_class(item['id'], (getattr(onto, term),))
        recursive_classmaker_down(item['id'])
    return 0

In [7]:
#Using classmaker to make FTC classes

for key, value in nary.items():
    if value['parent'] == []:
        with onto:
            nc = types.new_class(key, (biological_process,))
        for item in value['children']:
            with onto:
                nc = types.new_class(item['id'], (getattr(onto, key),))
                recursive_classmaker_down(item['id'])
        


In [8]:
list(onto.classes())

[onto.therapeutic_agent,
 onto.biological_process,
 onto.molecular_function,
 onto.drug,
 onto.biological_cess,
 onto.behavior,
 onto.cell_adhesion,
 onto.biological_adhesion,
 onto.molecular_func,
 onto.binding,
 onto.acyl_binding,
 onto.alkene_binding,
 onto.something_binding,
 onto.ion_binding,
 onto.cation_binding,
 onto.anion_binding]

In [9]:
onto.save(file = "t2test3", format = "rdfxml")